In [4]:
!pip install transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 18.5MB/s 
     |████████████████████████████████| 890kB 20.1MB/s 
     |████████████████████████████████| 2.9MB 29.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9e4eff87fc0b6b3ad70f82bc89efe5d983189a77d0769bd84306dbda9c9220f3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [71]:
from transformers import MarianTokenizer, MarianMTModel
from typing import List

model_name = f'Helsinki-NLP/opus-mt-{"fr"}-{"en"}'
model = MarianMTModel.from_pretrained(model_name)               #Loads the MarianMT model
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_to_english(text): #Translation function
  if (not text):
    return [text]
  batch = tokenizer.prepare_seq2seq_batch(src_texts=[text])  #Creates word batches for the sequential passing to the model
  generate_model = model.generate(**batch)  
  translation: List[str] = tokenizer.batch_decode(generate_model, skip_special_tokens=True)  #Translation
  return translation

In [78]:
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
from langdetect import detect 
from googletrans import Translator
from requests.exceptions import ConnectionError



import requests
import csv
import json 



def get_startups(urls):

    startups=[]
    urlwebsite=[]
    Description=[]

    df=pd.DataFrame()
    url = requests.post(urls)
    soup = BeautifulSoup(url.content, 'html.parser')

    k=0
    for link in soup.select('a.mx-auto'):
            site=get_site(link['href'])
            startups.append(link['title'].split(" ")[-1])
            urlwebsite.append(site)
            Description.append(get_Description(site))
            k+=1
            # if(k>30):
            #      break

    df["Name"]=startups
    df["urlwebsite"]=urlwebsite
    df["Description"]=Description

    return df
    
def get_Description(url):
    try:
        page = requests.get(url) 
    except ConnectionError:
          try:
              page = requests.get(url.replace("https","http"))
          except: 
              return None 
    except:
          return None
          

    soup = BeautifulSoup(page.content, 'html.parser')  #Parse html code
    
    texts = soup.find("meta", property="og:description")        
    
    if (texts):      
      text_from_html=texts['content']

    else :
      metas = soup.find_all('meta') 
      meta= ([ meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description' ])
      if(len(meta)==0):
        return None
      text_from_html=meta[0]

    text_from_html=translate_to_english(text_from_html)[0]  

    return text_from_html     

    #except Exception as e:
    #print(e)

def get_site(urls):
    url = requests.post(urls)
    soup = BeautifulSoup(url.content, 'html.parser')

    return soup.select('a.urlwebsite')[0]['href'].split('?')[0]


urls="https://www.jaimelesstartups.fr/liste-des-startups-en-france/"


df=get_startups(urls)
df.to_csv("data.csv")
df

,Name,urlwebsite,Description
0,Green,https://www.mobilitytechgreen.com/,"As the leader of B2B carsharing in Europe, Mob..."
1,Bean's,https://www.beanspro.fr,None
2,Prevision,https://prevision.io/,None
3,Techinclic,https://techinclic.fr/,"A computer problem? We accompany you at home, ..."
4,PROTEKTOOL,https://protektool.com/,Avoid touching surfaces contaminated by viruse...
...,...,...,...
495,OtherOffice,https://www.otheroffice.fr,Specializing in Corporate Real Estate Strategy...
496,KARMA,https://karma.jobs/,None
497,Liebr,https://liebr.io/,"With Liebr, find the best mechanic for your mo..."
498,Naturicious,https://www.naturicious.com/fr/,"Naturicious CBD FRANCE, a French producer. Buy..."
